## Teoria dos Grafos

#### Trabalho 1

In [1]:
import numpy as np
import bisect as bis
import heapq as hpq
from sys import getsizeof
from collections import deque

In [2]:
bool_to_int = lambda x: 1 if x else 0
b2i = bool_to_int

In [3]:
def getIndex(L,x):
    i = 0
    while L[i] != x:
        i += 1
    return i        

In [4]:
def getSrtdSeqMedian(seq):
    n = len(seq)
    if n % 2:
        return seq[n // 2]
    else:
        return (seq[n // 2] + seq[(n // 2) - 1]) / 2

###### o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o ---- o 

# Objetivos

## Classe dos grafos

#### 1) Ler um grafo de um arquivo de texto [ Grafos Simples - Grafos com Peso ]

#### 2) Gerar um arquivo de texto com as seguintes informaçoes:

- Numero de Vértices;  
- Numero de Arestas;  
- Grau Mínimo, Grau Máximo, Grau Médio e Mediana de Grau;  
- Informações sobre as componentes conexas;  

#### 3) Representar os grafos como:

- Matriz de Adjacências;  
- Lista de Adjacências;  

#### 4) Busca em grafos:

- BFS;
- DFS;

####  5) Calcular a Distância, Diâmetro e Pseudo-diâmetro:

####  6) Dijsktra
####  7) MST
####  8) Excentricidade

In [5]:
class AbstractGraph:
    def __init__(self, filename):
        f = open(filename, 'r')

        n_nodes = int(f.readline())

        self._initialize(n_nodes)

        self.n_nodes = n_nodes
        self.n_edges = 0

        for l in f:
            self._update(l)

        f.close()

        self._finalize()

    def _initialize(self, n_nodes):
        self.graph = self._emptygraph(n_nodes)

    def _update(self, l):
        v,u = l.split()
        v = int(v)
        u = int(u)
        self._addedge(v, u)

    def _getdegrees(self):
        degrees = []

        for v in range(self.n_nodes):
            d = self._getdegree(v + 1)
            bis.insort(degrees, d)

        return degrees

    def _finalize(self):
        self._savedegreeinfo()

    def _savedegreeinfo(self):
        degrees = self._getdegrees()

        self.degree_min    = degrees[0]
        self.degree_median = getSrtdSeqMedian(degrees)
        self.degree_max    = degrees[-1]
        self.degree_mean   = 2*self.n_edges/self.n_nodes

    def _writedegreeinfo(self, f):
        f.write('Número de vértices = {}\n'.format(self.n_nodes))
        f.write('Número de arestas  = {}\n'.format(self.n_edges))
        f.write('Grau mínimo        = {}\n'.format(self.degree_min))
        f.write('Grau mediano       = {}\n'.format(self.degree_median))
        f.write('Grau máximo        = {}\n'.format(self.degree_max))
        f.write('Grau médio         = {}\n'.format(self.degree_mean))

    def BFS(self, root, filename = None):
        ' breadth first search '
        n_nodes = self.n_nodes

        parent = np.full(n_nodes, -1, int)
        level  = np.full(n_nodes, -1, int)

        queue = deque()
        queue.append(root)
        
        level[root - 1] = 0

        while len(queue) >= 1:
            v = queue.popleft()
            neighbors = self._getneighbors(v)
            for u in neighbors:
                if level[u - 1] == -1:
                    level[u - 1] = level[v - 1] + 1
                    parent[u - 1] = v
                    queue.append(u)

        if filename:

             with open(filename, 'w') as f:

                bfs_tree = [(i,j) for i,j in zip(parent,level)]
                f.write(repr(n_nodes) + '\n')
                for node in bfs_tree:

                    f.write(repr(node) + '\n')

        return parent,level
    

    def DFS(graph, root, filename = None):
        ' depth first search '
        
        n_nodes = graph.n_nodes

        parent = np.full(n_nodes, -1, int)
        level  = np.full(n_nodes, -1, int)

        discovered = np.full(n_nodes, False)
        stack = [root]

        discovered[root - 1] = True
        level[root - 1] = 0

        while len(stack) >= 1:
            v = stack.pop(0)
            for u in range(n_nodes)[::-1]:
                if graph._isedge(v, u + 1) and not discovered[u]:
                    discovered[u] = True
                    stack.append(u + 1)


        if filename:

             with open(filename, 'w') as f:

                dfs_tree = [(i,j) for i,j in zip(parent,level)]
                f.write(repr(n_nodes) + '\n')
                for node in dfs_tree:

                    f.write(repr(node) + '\n')

        return parent,level

    def ConnectedComponents(self, filename = None):
        discovered = 0

        n_nodes = self.n_nodes

        parent    = np.full(n_nodes, -1, int)
        level     = np.full(n_nodes, -1, int)
        component = np.full(n_nodes, -1, int)
        components = []

        while discovered < n_nodes:
            root = getIndex(component, -1)

            queue = deque()
            queue.append(root + 1)

            level[root] = 0
            component[root] = root

            discovered += 1

            this = deque()
            this.append(root)

            while len(queue) >= 1:

                v = queue.popleft()

                neighbors = self._getneighbors(v)
                for u in neighbors:
                        if level[u - 1] == -1:
                            discovered   += 1
                            level[u - 1]     = level[v - 1] + 1
                            parent[u - 1]    = v
                            component[u - 1] = root
                            this.append(u - 1)
                            queue.append(u)

            this.appendleft(len(this))
            bis.insort(components, this)

        if filename:

             with open(filename, 'w') as f:
                f.write(str(len(components)) + '\n \n')
                for c in components[::-1]:
                    for l in c:
                        f.write(str(l) + '\n')
                    f.write('\n')
                f.write('\n')

        return discovered, parent, level, component

    def Diameter(self):

        diameter = 0

        for v in range(self.n_nodes):

            temp     = max(self.BFS(v+1)[1])
            diameter = max(temp, diameter)

        return diameter

    def PseudoDiameter(self):

        v = np.random.randint(1,self.n_nodes+1)

        bfs_v = self.BFS(v)[1]

        u = np.argmax(bfs_v) + 1
        pseudo_diam = [0,0,bfs_v[u - 1]]

        while pseudo_diam[-1] > pseudo_diam[-2] and pseudo_diam[-2] >= pseudo_diam[-3]:

            bfs_u = self.BFS(self,u)[1]
            u     = np.argmax(bfs_u) + 1
            pseudo_diam += [bfs_u[u - 1]]

        return pseudo_diam[-1]

    def save(self, filename):
        f = open(filename, 'w')

        self._writedegreeinfo(f)

        f.close()

In [6]:
class ArrayGraph(AbstractGraph):  
    
    def _emptygraph(self, n_nodes):
        return np.full((n_nodes, n_nodes), False, dtype=bool)
    
    def _getdegree(self, v): # Use _isedge
        d = 0
        
        for u in range(self.n_nodes):
            if self._isedge(v, u + 1):
                d += 1
        
        return d
    
    def _addedge(self, v, u):
        if not (self.graph[v - 1, u - 1] and self.graph[u - 1, v - 1]):
            self.graph[v - 1, u - 1] = True
            self.graph[u - 1, v - 1] = True
            self.n_edges += 1

    def _isedge(self, v, u):
        return self.graph[v - 1, u - 1] and self.graph[v - 1, u - 1]
    
    def _getneighbors(self, v): # Use _isedge
        return [u + 1 for u in range(self.n_nodes) if self._isedge(v, u + 1)]
    

In [7]:
class ListGraph(AbstractGraph):

    def _emptygraph(self, n_nodes):
        return [[] for _ in range(n_nodes)]
    
    def _addedge(self, v, u):            
        v_edges = self.graph[v - 1]
        u_edges = self.graph[u - 1]

        if not self._isedge(v, u):
            bis.insort(v_edges, u)
            bis.insort(u_edges, v)
            self.n_edges += 1

    def _finalize(self):
        self._casttondarray()
        self._savedegreeinfo()
    
    def _casttondarray(self):
        for i in range(len(self.graph)):
            self.graph[i] = np.array(self.graph[i])
        self.graph = np.array(self.graph)
    
    def _getdegree(self, v):
        d = len(self.graph[v - 1])
        return d
    
    def _isedge(self, v, u):
        return (u in self.graph[v - 1]) and (v in self.graph[u - 1])
    
    def _getneighbors(self, v):
        return self.graph[v - 1]

In [8]:
class AbstractWeightedGraph(AbstractGraph):
    
    def _update(self, l):
        v,u,w = l.split()
        v = int(v)
        u = int(u)
        w = float(w)
        self._addedge(v, u, w)
        
    def _initialize(self, n_nodes):
        self.graph   = self._emptygraph(n_nodes)
        self.weights = self._emptyweights(n_nodes)
    
    def Dijkstra(self, root):
        n_nodes = self.n_nodes

        distance = np.full(n_nodes, np.Inf, float)
        explored = np.full(n_nodes, False, bool)
        priority_queue = []

        distance[root - 1] = 0
        hpq.heappush(priority_queue, (0, root))
    
        while len(priority_queue) >= 1:
            d, v = hpq.heappop(priority_queue)
            if not explored[v - 1]:
                explored[v - 1] = True
                neighbors = self._getneighbors(v)
                for u in neighbors:
                    if distance[u - 1] > distance[v - 1] + self._getweight(v, u):
                        distance[u - 1] = distance[v - 1] + self._getweight(v, u)
                        hpq.heappush(priority_queue, (distance[u - 1], u))
            else:
                print("u r dumb")

        return distance

    def Eccentricity(self, root):
        # Do Bellman-Ford then take max
        n_nodes = self.n_nodes

        distance = np.full(n_nodes, np.Inf, float)
        distance[root - 1] = 0.
        for length in range(1, n_nodes):
            for v in range(1, n_nodes + 1):
                neighbors = self._getneighbors(v)
                for u in neighbors:
                    distance[v - 1] = min(distance[v - 1], distance[u - 1] + self._getweight(v, u))

        return max(distance)
    
    def BFS(self, root, filename = None):
        return RaiseException("Undefined for Weighted Graphs.")
    
    def DFS(self, root, filename = None):
        return RaiseException("Undefined for Weighted Graphs.")

In [9]:
class WeightedArrayGraph(ArrayGraph, AbstractWeightedGraph):

    def _emptygraph(self, n_nodes):
        return np.full((n_nodes, n_nodes), np.NaN, dtype=float)

    def _emptyweights(self, n_nodes):
        return self.graph

    def _addedge(self, v, u, w):
        if not self._isedge(v, u):
            self.graph[v - 1, u - 1] = w
            self.graph[u - 1, v - 1] = w
            self.n_edges += 1

    def _isedge(self, v, u):
        return (not np.isnan(self.graph[v - 1, u - 1])) and (not np.isnan(self.graph[v - 1, u - 1]))

    def _getweight(self, v, u):
        return self.weights[v - 1, u - 1]

In [10]:
class WeightedListGraph(ListGraph, AbstractWeightedGraph):

    def _emptygraph(self, n_nodes):
        return [[] for _ in range(n_nodes)]
    
    def _emptyweights(self, n_nodes):
        return [[] for _ in range(n_nodes)]
    
    def _addedge(self, v, u, w):
        v_edges = self.graph[v - 1]
        u_edges = self.graph[u - 1]
            
        v_weights = self.weights[v - 1]
        u_weights = self.weights[u - 1]

        if not self._isedge(v, u):
            bis.insort(v_edges, u)
            bis.insort(u_edges, v)
        self.n_edges += 1

        u_idx = getIndex(v_edges, u)
        v_idx = getIndex(u_edges, v)

        v_weights.insert(u_idx, w)
        u_weights.insert(v_idx, w)

    def _finalize(self):
        self._casttondarray()
        self._savedegreeinfo()
    
    def _casttondarray(self):
        for i in range(len(self.graph)):
            self.graph[i]   = np.array(self.graph[i])
            self.weights[i] = np.array(self.weights[i])
        self.graph   = np.array(self.graph)
        self.weights = np.array(self.weights)
        
    def _getweight(self, v, u):
        if self._isedge(v, u):
            u_idx = getIndex(self._getneighbors(v), u)
            return self.weights[v - 1][u_idx]
        else:
            RaiseException("`v` and `u` aren't neighbors.")

In [19]:
A = WeightedListGraph("wtest.txt")
B = WeightedArrayGraph("wtest.txt")

In [20]:
print(A.graph)
print(A.weights)

[array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]) array([1, 4, 5, 7])
 array([1, 5, 6, 8]) array([1, 2, 7, 9]) array([1, 2, 3]) array([1, 3])
 array([1, 2, 4]) array([1, 3]) array([1, 4]) array([1])]
[array([ 4. ,  2. ,  0.5,  5. ,  4. ,  6. ,  3. , -1. , 55. ])
 array([ 4.,  5., -1.,  3.]) array([ 2., 11.,  8.,  0.])
 array([0.5, 5. , 3. , 1. ]) array([ 5., -1., 11.]) array([4., 8.])
 array([6., 3., 3.]) array([3., 0.]) array([-1.,  1.]) array([55.])]


In [21]:
print(B.graph)
print(B.weights)

[[ nan  4.   2.   0.5  5.   4.   6.   3.  -1.  55. ]
 [ 4.   nan  nan  5.  -1.   nan  3.   nan  nan  nan]
 [ 2.   nan  nan  nan 11.   8.   nan  0.   nan  nan]
 [ 0.5  5.   nan  nan  nan  nan  3.   nan  1.   nan]
 [ 5.  -1.  11.   nan  nan  nan  nan  nan  nan  nan]
 [ 4.   nan  8.   nan  nan  nan  nan  nan  nan  nan]
 [ 6.   3.   nan  3.   nan  nan  nan  nan  nan  nan]
 [ 3.   nan  0.   nan  nan  nan  nan  nan  nan  nan]
 [-1.   nan  nan  1.   nan  nan  nan  nan  nan  nan]
 [55.   nan  nan  nan  nan  nan  nan  nan  nan  nan]]
[[ nan  4.   2.   0.5  5.   4.   6.   3.  -1.  55. ]
 [ 4.   nan  nan  5.  -1.   nan  3.   nan  nan  nan]
 [ 2.   nan  nan  nan 11.   8.   nan  0.   nan  nan]
 [ 0.5  5.   nan  nan  nan  nan  3.   nan  1.   nan]
 [ 5.  -1.  11.   nan  nan  nan  nan  nan  nan  nan]
 [ 4.   nan  8.   nan  nan  nan  nan  nan  nan  nan]
 [ 6.   3.   nan  3.   nan  nan  nan  nan  nan  nan]
 [ 3.   nan  0.   nan  nan  nan  nan  nan  nan  nan]
 [-1.   nan  nan  1.   nan  nan  nan  nan  na

In [27]:
A.Dijkstra(1)

u r dumb
u r dumb
u r dumb
u r dumb
u r dumb
u r dumb


array([-2.,  2.,  2.,  0.,  3.,  4.,  3.,  2., -1., 55.])

In [23]:
A.Dijkstra(4)

u r dumb
u r dumb
u r dumb
u r dumb
u r dumb
u r dumb


array([-1.5,  2.5,  2.5,  0. ,  3.5,  4.5,  3. ,  2.5, -0.5, 55.5])

In [ ]:
[(A.Eccentricity(i),B.Eccentricity(i)) for i in range(10)]

In [ ]:
assert(False)

## Comparação entre as Representações :

- getSizeOf(asgraph.array) = 1048788337 bytes = 1.04 GB  em 307.8 Segundos 
- getSizeOf(asgraph.list) = 259176 bytes = 0.25 MB em 0.87 Segundos  
- getSizeOf(dblp.array) - 208 GB*  
- getSizeOf(dblp.list) - 50 MB em 534.43 Segundos
- getSizeOf(livejournal.array) - 1.5 TB*  
- getSizeOf(livejournal.list) - 350 MB*

In [ ]:
from time import clock

In [ ]:
y0io

In [ ]:
t_0 = clock()
List_as_graph = ListGraph('as_graph.txt')
clock() - t_0

In [ ]:
t_0 = clock()
List_graph_dblp = ListGraph('dblp.txt')
clock() - t_0


## Tempos de Explorações

### as_graph.txt execução em 1000 vértices aleatórios:
- BFS.lista - 117.35 segundos - Tempo Médio = 0.117s  
- DFS.lista - 207.7 segundos  - Tempo Médio = 0.208s

In [ ]:
np.random.seed(1)
Random_Vertices = np.random.randint(1,List_as_graph.n_nodes+1,1000)

In [ ]:
t_0 = clock()
Array_as_graph = ArrayGraph('as_graph.txt')
clock() - t_0

In [ ]:
t_0 = clock()

for v in Random_Vertices:
    List_as_graph.BFS(v)
    
print(clock() - t_0)

In [ ]:
t_0 = clock()

for v in Random_Vertices:
    List_as_graph.DFS(v)
    
print(clock() - t_0)

### dblp.txt execução em 1000 vértices aleatórios:

In [ ]:
np.random.seed(1)
Random_Vertices = np.random.randint(1,List_graph_dblp.n_nodes+1,1000)

In [ ]:
t_0 = clock()

for v in Random_Vertices:
    List_graph_dblp.BFS(v)
    
print(clock() - t_0)

In [ ]:
t_0 = clock()

for v in Random_Vertices:
    List_graph_dblp.DFS(v)
    
print(clock() - t_0)

In [ ]:
t_0 = clock()
L = List_as_graph.Pseudo_diameter()    
print(clock() - t_0)

In [ ]:
t_0 = clock()
L = List_graph_dblp.Pseudo_diameter()    
print(clock() - t_0)

## Distância entre os pontos (10,20), (10,30) e (20,30)

In [ ]:
%timeit List_as_graph.Distance(10,20)

In [ ]:
%timeit List_as_graph.Distance(10,30)

In [ ]:
%timeit List_as_graph.Distance(20,30)

In [ ]:
List_as_graph.Distance(10,20), List_as_graph.Distance(10,30), List_as_graph.Distance(20,30)

In [ ]:
%timeit List_graph_dblp.Distance(10,20)

In [ ]:
%timeit List_graph_dblp.Distance(10,30)

In [ ]:
%timeit List_graph_dblp.Distance(20,30)

## Componentes Conexas


In [ ]:
%timeit List_as_graph.BFS(1)

In [ ]:
%timeit List_as_graph.BFS(2)

In [ ]:
cc = List_as_graph.BFS(3)

In [ ]:
cc[0][:30]

In [ ]:
aa = List_graph_dblp.BFS(1)[0][:30]

In [ ]:
aa

In [ ]:
bb = List_graph_dblp.BFS(2)[0][:30]

In [ ]:
bb

In [ ]:
cc =  List_graph_dblp.BFS(3)[0][:30]

In [ ]:
cc

In [ ]:
qtas_comps2, len(maior2), len(menor2)

In [ ]:
t_0 = clock()
BigL = ListGraph('live_journal.txt')
clock() - t_0

## Diâmetro dos Grafos

#### as_graph.txt:

In [ ]:
%time ListGraph('dblp.txt')

In [ ]:
%time qtas_comps2, maior2, menor2 =List_as_graph.ConnectedComponents()

# Trabalho 2